# Step 2: Private Prediction using Syft Keras (Server)

In [1]:
#import torch
import syft
# hook = syft.TorchHook(torch)
# alice = syft.VirtualWorker(hook, id="alice")  # alice computes on behalf of data owner
# bob = syft.VirtualWorker(hook, id="bob")      # bob computes on behalf of model owner
# carol = syft.VirtualWorker(hook, id="carol")  # carol acts as crypto producer

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU

In [4]:
task_classes = 10
task_shape = [None, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [5]:
model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=task_shape[1:]))
model.add(AveragePooling2D((2, 2)))
model.add(ReLU())
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(ReLU())
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(ReLU())
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [7]:
data = np.ones([1, 28, 28, 1])

In [8]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[ 10.450904   -5.304683    1.5378038  -4.0573554  -0.8924685  -1.6893905
   17.186888  -18.903719    7.104549   -5.302046 ]]


### Secure the model by sharing the weights

In [9]:
# prot = tfe.SecureNN(player_0=alice,
#                     player_1=bob,
#                     comparison_helper=carol,
#                     crypto_producer=carol)

tfe_model = model.share(prot=None)

In [10]:
private_prediction = tfe_model.predict(data)
print("Private Prediction:", private_prediction)

Private Prediction: [[ 10.450904   -5.304683    1.5378038  -4.0573554  -0.8924685  -1.6893905
   17.186888  -18.903719    7.104549   -5.302046 ]]


In [11]:
np.testing.assert_allclose(private_prediction, public_prediction, rtol=0.001)